# 💌 通用垃圾訊息分類器 | Universal Spam Classifier (SMS & Email)

歡迎來到通用垃圾訊息分類器的實作筆記本！

這份 Colab 筆記本將帶您打造一個強大的**通用垃圾訊息分類器**，它不僅能處理簡訊 📱，也能處理電子郵件 📧！

我們將展示從資料探索、文字雲視覺化，到模型訓練與建立一個即時預測系統的完整流程。 ✨

## 🛠️ Setup: 載入必要的函式庫

首先，我們需要載入所有會用到的 Python 函式庫，包含資料處理、視覺化和機器學習等工具。

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import nltk
from nltk.corpus import stopwords
import string
import os

# Scikit-learn 相關工具
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

# 下載 NLTK 的停用詞資料庫
nltk.download('stopwords')

print("✅ Setup Complete: 所有函式庫都已準備就緒！")

# Part 1: 簡訊垃圾訊息分類 (SMS Spam Classification) 💬

### 1.1. 資料載入與清理
我們將從 Kaggle 的公開資料集 `spam.csv` 開始。這個步驟包含讀取資料、整理欄位、將標籤轉換為數字格式（`ham` -> 0, `spam` -> 1），並移除重複的訊息。

In [ ]:
# 從 GitHub raw URL 讀取資料
url = 'https://raw.githubusercontent.com/vvchung/Universal_Spam_Classifier/master/spam.csv'
df = pd.read_csv(url, encoding='latin1')

# --- 資料清理 ---
# 1. 只保留需要的欄位
df = df[['v1', 'v2']]
# 2. 重新命名欄位，使其更具可讀性
df.rename(columns={'v1': 'target', 'v2': 'text'}, inplace=True)
# 3. 將文字標籤轉換為數字
df['target'] = LabelEncoder().fit_transform(df['target'])
# 4. 移除重複的資料
df = df.drop_duplicates(keep='first')

print(f"資料集清理完成！目前資料形狀: {df.shape}")
df.head()

### 1.2. 探索性資料分析 (EDA)
在建立模型之前，讓我們先來深入了解資料！我們會分析正常訊息和垃圾訊息的比例，並透過漂亮的文字雲 ☁️ 來看看兩者最常用的詞彙有什麼不同。

In [ ]:
# 1. 查看類別分佈 (Class Distribution)
plt.figure(figsize=(6, 6))
df['target'].value_counts().plot(kind='pie', autopct='%1.1f%%', labels=['Ham (正常)', 'Spam (垃圾)'], colors=['skyblue', 'salmon'])
plt.title('訊息類別分佈')
plt.ylabel('') # 隱藏 y 軸標籤
plt.show()

# 2. 建立文字預處理函式
def preprocess_text(text):
    # 轉為小寫
    text = text.lower()
    # 移除標點符號
    text = ''.join([char for char in text if char not in string.punctuation])
    # 分詞並移除停用詞
    words = text.split()
    words = [word for word in words if word not in stopwords.words('english')]
    return ' '.join(words)

df['processed_text'] = df['text'].apply(preprocess_text)

# 3. 產生文字雲 (Word Clouds)
wc = WordCloud(width=800, height=400, min_font_size=10, background_color='white')

# 垃圾訊息文字雲
spam_corpus = ' '.join(df[df['target'] == 1]['processed_text'].tolist())
spam_wc = wc.generate(spam_corpus)
plt.figure(figsize=(12, 6))
plt.imshow(spam_wc)
plt.title('垃圾訊息 (Spam) 中最常見的詞彙 ☁️')
plt.axis('off')
plt.show()

# 正常訊息文字雲
ham_corpus = ' '.join(df[df['target'] == 0]['processed_text'].tolist())
ham_wc = wc.generate(ham_corpus)
plt.figure(figsize=(12, 6))
plt.imshow(ham_wc)
plt.title('正常訊息 (Ham) 中最常見的詞彙 ☁️')
plt.axis('off')
plt.show()

### 1.3. 模型建立與評估
現在，我們要將處理過的文字轉換成機器可以理解的數字格式（向量化），然後訓練一個`多項式樸素貝氏 (Multinomial Naive Bayes)` 分類器，這是一個非常適合處理文字分類問題的經典模型。

In [ ]:
# 定義特徵 (X) 和目標 (y)
X = df['processed_text']
y = df['target']

# 將資料集分割為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用 CountVectorizer 將文字轉換為詞頻向量
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# 訓練模型
model = MultinomialNB()
model.fit(X_train_counts, y_train)

# 在測試集上進行預測與評估
y_pred = model.predict(X_test_counts)
print("--- 💬 SMS 分類器成果 ---")
print(f"模型準確率 (Accuracy): {accuracy_score(y_test, y_pred) * 100:.2f}%")

# 繪製混淆矩陣 (Confusion Matrix)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='viridis', xticklabels=['Ham', 'Spam'], yticklabels=['Ham', 'Spam'])
plt.title('SMS 分類器混淆矩陣')
plt.xlabel('預測標籤')
plt.ylabel('真實標籤')
plt.show()

# Part 2: 郵件垃圾訊息分類 (Email Spam Classification) 📂

### 2.1. 載入 Email 資料集
在這個部分，我們將展示如何處理另一種常見的資料格式：由大量獨立文字檔組成的資料集。我們將下載 Ling-Spam 語料庫並編寫一個函式來讀取它。

In [ ]:
# 下載並解壓縮 Ling-Spam 資料集
!wget https://raw.githubusercontent.com/vvchung/Universal_Spam_Classifier/main/lingspam_public.tar.gz -O lingspam_public.tar.gz
!tar -xzf lingspam_public.tar.gz

def load_email_data(base_path, folder_indices):
    texts, labels = [], []
    for i in folder_indices:
        folder_path = os.path.join(base_path, f'part{i}')
        # 確保資料夾存在
        if not os.path.exists(folder_path):
            continue
        for filename in os.listdir(folder_path):
            label = 1 if filename.startswith('spmsg') else 0
            with open(os.path.join(folder_path, filename), 'r', errors='ignore') as f:
                texts.append(f.read())
                labels.append(label)
    return texts, labels

# 定義訓練集和測試集的路徑（9 個資料夾用於訓練，1 個用於測試）
DATA_DIR = 'lingspam_public/bare' # 使用最原始的文字版本
email_X_train, email_y_train = load_email_data(DATA_DIR, list(range(1, 10)))
email_X_test, email_y_test = load_email_data(DATA_DIR, [10])

print(f"📧 Email 資料載入完成！")
print(f"訓練資料筆數: {len(email_X_train)}")
print(f"測試資料筆數: {len(email_X_test)}")

### 2.2. 訓練 Email 分類器
對於 Email 資料，我們使用 `TfidfVectorizer`。與 `CountVectorizer` 不同，TF-IDF 不僅考慮詞頻，還會考慮一個詞在所有文件中的普遍程度，這對於較長的文件（如郵件）通常效果更好。

In [ ]:
# 使用 TfidfVectorizer 將 Email 文字轉換為 TF-IDF 向量
tfidf_vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
email_X_train_tfidf = tfidf_vectorizer.fit_transform(email_X_train)
email_X_test_tfidf = tfidf_vectorizer.transform(email_X_test)

# 訓練一個新的 Naive Bayes 模型
email_model = MultinomialNB()
email_model.fit(email_X_train_tfidf, email_y_train)
email_y_pred = email_model.predict(email_X_test_tfidf)

print("--- 📂 Email 分類器成果 ---")
print(f"模型準確率 (Accuracy): {accuracy_score(email_y_test, email_y_pred) * 100:.2f}%")

# Part 3: 終極預測系統 (Unified Predictive System) 🎯

最後，我們來打造一個真正實用的系統！我們將使用在 Part 1 中訓練好的 SMS 模型 (`model`) 和向量化工具 (`vectorizer`)，因為它的預處理流程最完整。這個函式可以接收任何文字輸入，並立即判斷它是否為垃圾訊息。

In [ ]:
def predict_message(text):
    """
    使用我們訓練好的 SMS 模型來預測任何新訊息。
    Args:
        text (str): 輸入的訊息 (SMS 或 Email)。
    Returns:
        str: 預測結果，'Spam (垃圾)' 或 'Not Spam (Ham) (正常)'。
    """
    # 1. 使用與訓練時相同的流程預處理文字
    processed_text = preprocess_text(text)

    # 2. 使用已訓練好的 vectorizer 轉換文字
    text_counts = vectorizer.transform([processed_text])

    # 3. 使用已訓練好的 model 進行預測
    prediction = model.predict(text_counts)[0]

    # 4. 回傳人類可讀的結果
    if prediction == 1:
        return "Spam (垃圾) 😠"
    else:
        return "Not Spam (Ham) (正常) 😊"

# --- 🚀 實戰測試 ---
print("--- 終極預測系統實戰 ---")

# 範例 1: 典型的垃圾簡訊
sms_spam = 'Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)'
print(f"輸入: '{sms_spam}'")
print(f"預測結果: {predict_message(sms_spam)}\n")

# 範例 2: 典型的垃圾郵件
email_spam = 'Subject: Low-cost Insurance. Are you paying too much for your car insurance? Get a free quote today and save up to 40%. Visit our website now!'
print(f"輸入: '{email_spam}'")
print(f"預測結果: {predict_message(email_spam)}\n")

# 範例 3: 正常的訊息
ham_message = 'Hi Sarah, just wanted to confirm our meeting for tomorrow at 10 AM. Let me know if that still works for you.'
print(f"輸入: '{ham_message}'")
print(f"預測結果: {predict_message(ham_message)}")